In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/24 21:31:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/24 21:31:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
from pyspark.ml.clustering import KMeans

In [ ]:
import pandas as pd
from datetime import date, datetime
import matplotlib.pyplot as plt

from sqlalchemy import create_engine, Column, Integer, Table, MetaData

In [ ]:
user = 'root'
password = 'secret'
db = 'db'
host = 'localhost'
engine = create_engine(f"postgresql://{user}:{password}@{host}:5432/{db}")

df = pd.read_sql_query(f'select * from data', con=engine)

In [ ]:
data = spark.createDataFrame(df)
data

In [ ]:
from pyspark.ml.feature import VectorAssembler
  
vec_assembler = VectorAssembler(inputCols=data.columns,
                                outputCol='features')
  
final_data = vec_assembler.transform(data)
final_data.select('features').show(5)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
for i in range(2,15):
    KMeans_algo=KMeans(k=i)    
    KMeans_fit=KMeans_algo.fit(final_data)
    output=KMeans_fit.transform(final_data)

    score=evaluator.evaluate(output)
    silhouette_score.append(score)
    
    print(f"Silhouette at {i}: {score:.3f}")

In [ ]:
plt.plot(list(range(2, 15)), silhouette_score)

In [ ]:
KMeans_algo=KMeans(k=7)    
KMeans_fit=KMeans_algo.fit(final_data)
output=KMeans_fit.transform(final_data)

In [ ]:
preds = output.select('prediction').collect()
preds = list(map(lambda item: item.asDict()['prediction'], preds))

In [ ]:
metadata = MetaData()
mytable = Table('data', metadata, autoload=True, autoload_with=engine)
new_column = Column('predicted_cluster', Integer)
alter_query = f"ALTER TABLE data ADD COLUMN predicted_cluster INTEGER"
# engine.execute(alter_query)

In [ ]:
update_query = mytable.update().values(predicted_cluster=preds)
engine.execute(update_query)

In [79]:
update_query = mytable.update().values(predicted_cluster=preds)
engine.execute(update_query)